# 作业第6周  歌词生成练习

# 导入库

In [1]:
#首先执行GPU资源分配代码，勿删除。
import GPU
# GPU.show()
GPU.alloc(0,1024)

Success.


# 导入需要的依赖

In [2]:
import tensorflow as tf 
from tensorflow import keras 
import random 
import zipfile
import numpy as np 
import math 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN,LSTM,GRU

# 加载训练集

In [3]:
with zipfile.ZipFile('data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
print(corpus_chars[:40])

想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每


# 2、建立字符索引
我们将每个字符映射成一个从0开始的连续整数，又称索引，来方便之后的数据处理。  
为了得到索引，我们将数据集里所有不同字符取出来，然后将其逐一映射到索引来构造词典。  
接着，打印 vocab_size，即词典中不同字符的个数，又称词典大小。  

In [4]:
#使用set()函数将数据集中重复的字符删掉，然后放入列表中。
idx_to_char=list(set(corpus_chars))
len(idx_to_char)

2583

In [5]:
#将字符映射到索引
char_to_idx = {char:i for i, char in enumerate(idx_to_char)}
vocab_size=len(char_to_idx)
vocab_size

2583

In [6]:
#将字符转化成索引
corpus_indices= [char_to_idx[char] for char in corpus_chars]
len(corpus_indices)

63282

corpus_indices 中是原数据集corpus_chars 中所有字符的索引值。我们可以打印前20个字符机器对应的索引：

In [7]:
sample=corpus_indices[1000:1020]
print('indices:', sample)
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))

indices: [139, 1794, 1763, 475, 300, 2065, 139, 2264, 2259, 2264, 2259, 2252, 1950, 2399, 1366, 139, 976, 300, 752, 1383]
chars: 
随风跟著我
一口一口吃掉忧愁
载著你 


# 3、时序数据的采样
在训练中我们需要每次随机读取小批量样本和标签。与其他实验数据不同的是，时序数据的一个样本通常包含连续的字符。假设时间步数为5，样本序列为5个字符，即“想”“要”“有”“直”“升”。该样本的标签序列为这些字符分别在训练集中的下一个字符，即“要”“有”“直”“升”“机”。我们有两种方式对时序数据进行采样，分别是随机采样和相邻采样。  
## 3.1 随机采样
下面的代码每次从数据里随机采样一个小批量。  
其中批量大小 batch_size 指每个小批量的样本数，num_steps为每个样本所包含的时间步数。   
在随机采样中，每个样本是原始序列上任意截取的一段序列。相邻的两个随机小批量在原始序列上的位置不一定相邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。在训练模型时，每次随机采样前都需要重新初始化隐藏状态。 
## 3.2 相邻采样
除对原始序列做随机采样之外，我们还可以令相邻的两个随机小批量在原始序列上的位置相邻。这时候，我们就可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。这对实现循环神经网络造成了两方面影响：一方面，在训练模型时，我们只需在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。   
为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。

In [8]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices=np.array(corpus_indices)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0: batch_size*batch_len].reshape((batch_size, batch_len))
    epoch_size= (batch_len-1) //num_steps
    for i in range(epoch_size):
        i=i*num_steps
        X=indices[:, i: i+num_steps]
        Y=indices[:, i+1: i+num_steps+1]
        yield X, Y
        
my_seq=list(range(30))
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  [[ 0  1  2  3  4  5]
 [15 16 17 18 19 20]] 
Y: [[ 1  2  3  4  5  6]
 [16 17 18 19 20 21]] 

X:  [[ 6  7  8  9 10 11]
 [21 22 23 24 25 26]] 
Y: [[ 7  8  9 10 11 12]
 [22 23 24 25 26 27]] 



# 4、定义模型
## 4.1 定义循环神经网络层 
Keras 的 Rnn 模块提供了循环神经网络的实现。下面构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层 rnn_layer，并对权重做初始化。
rnn_layer 的输入形状为 (时间步数, 批量大小, 词典大小)，在前向计算后会分别返回输出和隐藏状态。  
其中输出指的是隐藏层在各个时间步上计算并输出的隐藏状态，它们通常作为后续输出层的输入。  
需要强调的是，该“输出”本身并不涉及输出层计算，形状为 (时间步数, 批量大小, 隐藏单元个数)。  
返回的隐藏状态指的是隐藏层在最后时间步的隐藏状态：当隐藏层有多层时，每一层的隐藏状态都会记录在该变量中。

In [9]:
num_hiddens = 256
batch_size = 160
num_steps = 35
model = Sequential()
model.add(keras.Input(batch_input_shape = (batch_size,num_steps))) 
model.add(Embedding(output_dim = 256,input_dim = vocab_size, input_length = num_steps))
#model.add(LSTM(units=num_hiddens,return_sequences=True, stateful=True))
model.add(GRU(units = num_hiddens,return_sequences = True, stateful = True))
model.add(Dense(units = vocab_size,activation = 'softmax' ))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (160, 35, 256)            661248    
                                                                 
 gru (GRU)                   (160, 35, 256)            394752    
                                                                 
 dense (Dense)               (160, 35, 2583)           663831    
                                                                 
Total params: 1,719,831
Trainable params: 1,719,831
Non-trainable params: 0
_________________________________________________________________


In [10]:
!pip install pydot graphviz
from tensorflow.keras.utils import plot_model 
plot_model(model = model,show_shapes = True)

This account is currently not available.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


将经过采样后的样本直接输入此网络中时，它经过的处理操作依次为：  
1. 输入形状为 (批量大小, 时间步数)  
2. 利用embedding编码得到 (批量大小, 时间步数, embedding大小  
3. 输入到 rnn 层，得到 (批量大小, 时间步数, 隐藏单元个数)
4. 经过 Dense 层，得到 (批量大小， 时间步数, 词典大小)  

In [11]:
def sample(preds, temperature = 1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predict_rnn_keras(prefix, num_chars):
    # 使用model的成员函数来初始化隐藏状态
    model.reset_states()
    output= [char_to_idx[prefix[0]]]  #上一次输出
    for t in range(num_chars + len(prefix) - 1):
        X = (np.array([output[-1]]).repeat(batch_size)).reshape((batch_size, 1))  #timestep=1
        Y = model(X)  # 前向计算
        if t < len(prefix) -1:
            output.append(char_to_idx[prefix[t+1]])   #引导前缀不使用预测结果
        else:
            output.append(sample(np.array(Y[0,0,:])))  #批量大小， 时间步数, 词典大小
            ##output.append(int(np.array(tf.argmax(Y,axis=-1))[0,0]))  #批量大小， 时间步数, 词典大小
    return''.join([idx_to_char[i] for i in output])

In [12]:
predict_rnn_keras('分开', 10)
#因为模型参数为随机值，所以预测结果也是随机的。

'分开榜喉陽剧犹鼠哼胖扇讨'

# 5、裁剪梯度

In [13]:
# 计算裁剪后的梯度示意代码，本例并不使用该函数！！！
def grad_clipping(grads,theta):
    norm=np.array([0]) 
    for i in range(len(grads)):
        norm+=tf.math.reduce_sum(grads[i] **2)
    norm=np.sqrt(norm).item()
    if norm <= theta:
        return grads
    
    new_gradient=[]
    for grad in grads:
        new_gradient.append(grad*theta/norm)
    return new_gradient

# 6、定义模型训练函数
初始化优化器

In [14]:
opt = keras.optimizers.Adam(learning_rate=1e-3, clipnorm=0.1)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# 7训练
定义梯度下降函数定义训练函数    
num_epochs：训练次数；   
batch_size：批次大小；   
pred_period：间隔多少次展示一次结果；   
pred_len：要求预测的字符长度。  

In [15]:
def train_and_predict_rnn_keras(num_epochs, batch_size, pred_period, pred_len, prefixes):
    for epoch in range(num_epochs):
        l_sum, n=0.0, 0
        model.reset_states()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps)
        for X, Y in data_iter:
            y_pred = model.train_on_batch(X,Y)     #y_pred[0]为loss， y_pred[1]为accuracy
            loss=y_pred[0]
            l_sum+=loss
            n+=1
            
        if (epoch+1) % pred_period==0:
            print('epoch %d, perplexity %f'% (epoch+1, math.exp(l_sum/n)))
            for prefix in prefixes:
                print('>>', predict_rnn_keras(prefix, pred_len))
                
num_epochs = 1000
train_and_predict_rnn_keras(num_epochs, batch_size, 100, 50, ['想要', '我们'])

epoch 100, perplexity 2.367327
>> 想要我看你不要
这里你再也只想很简单
不想再见
训导处报告 快使用双截棍 
生命中只是我和你融化了开
只
>> 我们忏悔
我的世界流浪
却又过去对医药箱说
别怪我
别怪我 屌  却想为了我要一点孤独
力会让半魔之前

epoch 200, perplexity 1.202386
>> 想要有直升机
想要打我在等动
用第一脸 在想念失去
你了把 长大的梦 的呦啊
我把天地下的每个人
都是我
>> 我们忏悔
生活习惯 这时候 不稀 
我知道是自己往前旁
木炭 剩一半
雨过蝴蝶
下大的国度性
就像是多汁
epoch 300, perplexity 1.068957
>> 想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每天在想想想想著你的笑
一
>> 我们忏悔
生死不想太多
会不会慢慢走开
为什么我连分开都迁就着你
我真的没有天份
安静的没这么快
我会学
epoch 400, perplexity 1.035919
>> 想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天我遇见你的微笑很久以
我在边功
>> 我们忏悔
生命珍贵 我用那十四行诗赞美
年轻的 一切
我只有一天的回忆
雨淋湿了天空
毁得很讲究
你说你
epoch 500, perplexity 1.024284
>> 想要有直升机
想要和你飞到宇宙去
想要和外婆参加  
吹着残忍住大
那有三光 训导处报告
三年二班周杰伦
>> 我们忏悔
生命珍贵 我用那十四行诗赞美
一起旅行
那童年的希望是
一台 时光机
你我翻滚过的榻榻米
味道
epoch 600, perplexity 1.020293
>> 想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每天在想想想想著你
这样的
>> 我们忏悔
生命珍贵 我用那十四行诗赞美
年轻的 一切
我们的爱 给的太快 
是从此就分的
让回忆皎洁
爱
epoch 700, perplexity 1.011737
>> 想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每天在想想想想著你
这样的
>> 我们忏悔
生命珍贵 我用那十四行诗赞美
年轻的 一切
我们在地平线感谢
爱停止向另一边倾斜
这首部曲的管

**注解：困惑度我们通常使用困惑度（perplexity）来评价语言模型的好坏。**  
困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；  
最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；  
基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。  
显然，任何一个有效模型的困惑度必须小于类别个数。  
在本例中，困惑度必须小于词典大小vocab_size。